\begin{equation}
\frac{d\rho}{dt} = -i[H(t), \rho] + \sum_k \left( c_k \rho c_k^\dagger - \frac{1}{2} \left\{ c_k^\dagger c_k, \rho \right\} \right)
\end{equation}

using a first-order Euler method, which approximates the density matrix at the next time step as:
\begin{equation}
\rho(t + \Delta t) \approx \rho(t) + \Delta t \cdot \frac{d\rho}{dt}
\end{equation}

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from qutip import *

# Parameters
gamma_m = 1
Omega_0 = 0.01 * gamma_m
sigma = 50 / gamma_m
top = 500 / gamma_m
gamma = 1.25e-4 * gamma_m
eta = 1
tau = 2.5 / gamma_m
dt = 0.01
Tc = 0.00005

theta_z1 = theta_x1 = +1.4
theta_z2 = theta_x2 = -0.40
tlist = np.arange(0, top, dt)
alpha = dt / tau


In [ ]:
omega_t = lambda t: Omega_0 * np.exp(-0.5 * ((t - top / 2) / sigma) ** 2)
delta_t = lambda t: 2*Omega_0 * (t / (top / 2) - 1)

plt.plot(tlist, omega_t(tlist))
plt.plot(tlist, delta_t(tlist))
plt.show()

In [ ]:
# Define Pauli operators
I, X, Z = qeye(2), sigmax(), sigmaz()

# Logical ⊗ 4 Gauge = 5 Qubits
N = 5
def op_on_qubit(op, i):
    return tensor([op if k == i else I for k in range(N)])

# Logical operators (qubit 0)
X_L = op_on_qubit(X, 0)
Z_L = op_on_qubit(Z, 0)

H = [[X_L, omega_t], [Z_L, delta_t]]


# Initial state: logical |0⟩ and gauge |0000⟩
zero = tensor([basis(2, 0)] * N)
one = X_L * zero
rho = ket2dm(zero)
psi0 = zero

c_ops = [np.sqrt(gamma) * op_on_qubit(X, i) for i in range(5)]


results = mesolve(H, ket2dm(psi0), tlist, c_ops=c_ops, e_ops=[zero*zero.dag(), one*one.dag()])

plt.plot(tlist, results.expect[0], label='0')
plt.plot(tlist, results.expect[1], label='1')
plt.legend()
plt.show()


In [ ]:
# Bacon-Shor Continuous QEC Simulation (Aligned with Phys. Rev. A 102, 022415)
theta_high = 0.5
theta_low = -0.5

def op_on_qubit(op, i):
    return tensor([op if k == i else I for k in range(N)])

# Define gauge operators (from Eq. (22))
G1 = op_on_qubit(Z, 1)
G2 = op_on_qubit(Z, 1) * op_on_qubit(Z, 2)
G3 = op_on_qubit(Z, 2)
G4 = op_on_qubit(Z, 3)
G5 = op_on_qubit(Z, 3) * op_on_qubit(Z, 4)
G6 = op_on_qubit(Z, 4)
G7 = op_on_qubit(X, 1)
G8 = op_on_qubit(X, 1) * op_on_qubit(X, 3)
G9 = op_on_qubit(X, 3)
G10 = op_on_qubit(X, 2)
G11 = op_on_qubit(X, 2) * op_on_qubit(X, 4)
G12 = op_on_qubit(X, 4)
G_reduced = [G1, G2, G3, G4, G5, G6, G7, G8, G9, G10, G11, G12]

Y = sigmay()

# Syndrome thresholding
thresholds = {
    (+1, +1, +1, -1): op_on_qubit(X, 3),
    (+1, +1, -1, -1): op_on_qubit(Y, 4),
    (+1, +1, -1, +1): op_on_qubit(Z, 4),
    (+1, -1, +1, +1): op_on_qubit(X, 1),
    (+1, -1, +1, -1): op_on_qubit(X, 2),
    (+1, -1, -1, -1): op_on_qubit(Y, 2),
    (+1, -1, -1, +1): op_on_qubit(Y, 1),
    (-1, -1, +1, +1): op_on_qubit(Y, 0),
    (-1, -1, +1, -1): op_on_qubit(Y, 3),
    (-1, -1, -1, -1): op_on_qubit(Y, 2),
    (-1, -1, -1, +1): op_on_qubit(Y, 1),
    (-1, +1, +1, +1): op_on_qubit(Z, 1),
    (-1, +1, +1, -1): op_on_qubit(Y, 1),
    (-1, +1, -1, -1): op_on_qubit(Y, 2),
    (-1, +1, -1, +1): op_on_qubit(Z, 2)
}

# Measurement + Evolution Setup
# -------------------------------
I_bar = [0.0] * 12
Cz1s, Cz2s, Cx1s, Cx2s = [], [], [], []
fidelities_0_L = []
fidelities_1_L = []

# Time evolution loop
for t in tlist:
    omega = omega_t(t)
    delta = delta_t(t)
    H = X_L * omega + Z_L * delta

    dWs = np.random.normal(scale=np.sqrt(dt), size=12)
    meas_terms = [np.sqrt(2 * gamma_m * eta) * (G * rho + rho * G - 2 * expect(G, rho) * rho) for G in G_reduced]

    # Master equation
    drho = -1j * (H * rho - rho * H)
    
    # the problem is here
    
    for G in G_reduced:
        drho += gamma_m * (G * rho * G - 0.5 * (G ** 2 * rho + rho * G ** 2))
        
    for m, dW in zip(meas_terms, dWs):
        drho += m * dW

    rho = (rho + dt * drho).unit()

    # Measurement currents
    I = [expect(G, rho) + dW / np.sqrt(dt) for G, dW in zip(G_reduced, dWs)]
    for i in range(12):
        I_bar[i] = (1 - alpha) * I_bar[i] + alpha * I[i]

    # Exponential filtering for 4 correlation signals
    decay = np.exp(-dt / Tc)
    weight = dt / Tc
    Cz1 = decay * (Cz1s[-1] if Cz1s else 0) + weight * (I_bar[0] * I_bar[1] * I_bar[2])
    Cz2 = decay * (Cz2s[-1] if Cz2s else 0) + weight * (I_bar[3] * I_bar[4] * I_bar[5])
    Cx1 = decay * (Cx1s[-1] if Cx1s else 0) + weight * (I_bar[6] * I_bar[7] * I_bar[8])
    Cx2 = decay * (Cx2s[-1] if Cx2s else 0) + weight * (I_bar[9] * I_bar[10] * I_bar[11])
    Cz1s.append(Cz1)
    Cz2s.append(Cz2)
    Cx1s.append(Cx1)
    Cx2s.append(Cx2)

    # Syndrome thresholding and error correction
    sx1 = +1 if Cx1 > theta_low else -1 if Cx1 < theta_high else 0
    sz1 = +1 if Cz1 > theta_low else -1 if Cz1 < theta_high else 0
    sx2 = +1 if Cx2 > theta_low else -1 if Cx2 < theta_high else 0
    sz2 = +1 if Cz2 > theta_low else -1 if Cz2 < theta_high else 0
    syndrome = (sx1, sz1, sx2, sz2)
    if syndrome in thresholds:
        correction = thresholds[syndrome]
        rho = correction * rho * correction.dag()

    # Fidelity
    one = X_L * zero
    fidelities_0_L.append(expect(ket2dm(zero), rho))
    fidelities_1_L.append(expect(ket2dm(one), rho))
    
    # Optional: print progress every 10 steps
    if int(t / dt) % 10 == 0:
        print(f"Progress: {int(t / dt) / len(tlist) * 100:.2f}%")

In [ ]:
plt.plot(tlist, fidelities_0_L, label='|0_L⟩')
plt.plot(tlist, fidelities_1_L, label='|1_L⟩')
# plt.ylim([0,0.1])
# plt.xlim([1,2])
plt.xlabel("Time")
plt.ylabel("Fidelity")
plt.legend()
plt.title("Logical State Fidelity under Continuous QEC with Correction")
plt.show()

In [ ]:
plt.plot(tlist, Cz1s, label='Cz1')
plt.plot(tlist, Cz2s, label='Cz2')
plt.plot(tlist, Cx1s, label='Cx1')
plt.plot(tlist, Cx2s, label='Cx2')

# also plot the threshholds lines theta
plt.axhline(theta_z1, color='r', linestyle='--', label='theta_z1')
plt.axhline(theta_z2, color='g', linestyle='--', label='theta_z2')
plt.axhline(theta_x1, color='b', linestyle='--', label='theta_x1')
plt.axhline(theta_x2, color='y', linestyle='--', label='theta_x2')

# plt.ylim([-0.5, 0.5])
plt.xlabel("Time")
plt.ylabel("Cz1, Cz2, Cx1, Cx2")
plt.legend()
plt.title("Measurement Currents")

In [ ]:
# Define gauge operators (from Eq. (22))
G1 = op_on_qubit(Z, 1)
G2 = op_on_qubit(Z, 1) * op_on_qubit(Z, 2)
G3 = op_on_qubit(Z, 2)
G4 = op_on_qubit(Z, 3)
G5 = op_on_qubit(Z, 3) * op_on_qubit(Z, 4)
G6 = op_on_qubit(Z, 4)
G7 = op_on_qubit(X, 1)
G8 = op_on_qubit(X, 1) * op_on_qubit(X, 3)
G9 = op_on_qubit(X, 3)
G10 = op_on_qubit(X, 2)
G11 = op_on_qubit(X, 2) * op_on_qubit(X, 4)
G12 = op_on_qubit(X, 4)
G_reduced = [G1, G2, G3, G4, G5, G6, G7, G8, G9, G10, G11, G12]

# Initialize lists for correlation signals
Cz1s, Cz2s, Cx1s, Cx2s = [], [], [], []
I_bar = [0.0] * len(G_reduced)

# Compute all computational basis states for a 5-qubit system
dim = 2**N
basis_states = [tensor([basis(2, int(b)) for b in format(i, '0{}b'.format(N))])
                for i in range(dim)]
# Initialize a list to store fidelities for all 32 basis states
fidelities_all = [[] for _ in range(dim)]

# Time evolution loop (stochastic simulation)
for t in tlist:
    omega = omega_t(t)
    delta = delta_t(t)
    H_current = X_L * omega + Z_L * delta

    dWs = np.random.normal(scale=np.sqrt(dt), size=len(G_reduced))

    meas_terms = [np.sqrt(2 * gamma_m * eta) * (G * rho + rho * G - 2 * expect(G, rho) * rho)
                  for G in G_reduced]

    # Hamiltonian evolution and dissipation
    drho = -1j * (H_current * rho - rho * H_current)
    for G in G_reduced:
        drho += gamma_m * (G * rho * G - 0.5 * (G**2 * rho + rho * G**2))
    for m, dW in zip(meas_terms, dWs):
        drho += m * dW
    # for c in c_ops:
    #     drho += c * rho * c.dag() - 0.5 * (c.dag() * c * rho + rho * c.dag() * c)

    rho = (rho + dt * drho).unit()

    # Measurement currents and filtering
    I = [expect(G, rho) + dW / np.sqrt(dt) for G, dW in zip(G_reduced, dWs)]
    for i in range(len(I)):
        I_bar[i] = (1 - alpha) * I_bar[i] + alpha * I[i]

    decay = np.exp(-dt / Tc)
    weight = dt / Tc
    Cz1 = decay * (Cz1s[-1] if Cz1s else 0) + weight * (I_bar[0] * I_bar[1] * I_bar[2])
    Cz2 = decay * (Cz2s[-1] if Cz2s else 0) + weight * (I_bar[3] * I_bar[4] * I_bar[5])
    Cx1 = decay * (Cx1s[-1] if Cx1s else 0) + weight * (I_bar[6] * I_bar[7] * I_bar[8])
    Cx2 = decay * (Cx2s[-1] if Cx2s else 0) + weight * (I_bar[9] * I_bar[10] * I_bar[11])
    Cz1s.append(Cz1)
    Cz2s.append(Cz2)
    Cx1s.append(Cx1)
    Cx2s.append(Cx2)

    # Fidelity tracking for all basis states
    for idx, state in enumerate(basis_states):
        fidelity = expect(ket2dm(state), rho)
        fidelities_all[idx].append(fidelity)

    # Optional: print progress every 10 steps
    if int(t / dt) % 10 == 0:
        print(f"Progress: {int(t / dt) / len(tlist) * 100:.2f}%")


In [ ]:
# Tc = 0.0000005
# plot Cz1, Cz2, Cx1, Cx2 as function of time
plt.plot(tlist, Cz1s, label='Cz1')
plt.plot(tlist, Cz2s, label='Cz2')
plt.plot(tlist, Cx1s, label='Cx1')
plt.plot(tlist, Cx2s, label='Cx2')
# also plot the threshholds lines theta
plt.axhline(theta_z1, color='r', linestyle='--', label='theta_z1')
plt.axhline(theta_z2, color='g', linestyle='--', label='theta_z2')
plt.axhline(theta_x1, color='b', linestyle='--', label='theta_x1')
plt.axhline(theta_x2, color='y', linestyle='--', label='theta_x2')

# plt.ylim([-0.5, 0.5])
plt.xlabel("Time")
plt.ylabel("Cz1, Cz2, Cx1, Cx2")
plt.legend()
plt.title("Measurement Currents")

In [ ]:
# Tc = 5
# plot Cz1, Cz2, Cx1, Cx2 as function of time
plt.plot(tlist, Cz1s, label='Cz1')
plt.plot(tlist, Cz2s, label='Cz2')
plt.plot(tlist, Cx1s, label='Cx1')
plt.plot(tlist, Cx2s, label='Cx2')
# also plot the threshholds lines theta
plt.axhline(theta_z1, color='r', linestyle='--', label='theta_z1')
plt.axhline(theta_z2, color='g', linestyle='--', label='theta_z2')
plt.axhline(theta_x1, color='b', linestyle='--', label='theta_x1')
plt.axhline(theta_x2, color='y', linestyle='--', label='theta_x2')

plt.ylim([-0.0005, 0.0005])
plt.xlabel("Time")
plt.ylabel("Cz1, Cz2, Cx1, Cx2")
plt.legend()
plt.title("Measurement Currents")

In [ ]:
# Example: plot fidelity for basis state corresponding to |00000>
for i in range(len(fidelities_all)):
    plt.plot(tlist, fidelities_all[i], label=f'Fidelity of {i} state')

plt.xlabel("Time")
plt.ylabel("Fidelity")
plt.legend()
plt.title("Fidelity for |00000⟩")
plt.show()